## Small utility to test migration status and deal with failed migrations
by removing them and trying again.
Should only be used when the migration failure reason has
been understood and solved. Or to reproduce a failure for
investigation


In [3]:
# this import is what's needed to work with CRAB+DBS client distribution in CMSSW
import CRABClient
from dbs.apis.dbsClient import DbsApi

In [24]:
# we migrate from G(lobal) to P(hys0)3
# bu which DBS instance do we migrate to ?
# Yuyi's dev machine
#migUrl='https://dbs3-test1.cern.ch/dbs/dev/global/DBSMigrate'
#phy3Url = 'https://dbs3-test1.cern.ch/dbs/dev/global/DBSReader'
# testbed
#migUrl='https://cmsweb-testbed.cern.ch/dbs/int/phys03/DBSMigrate'
#phy3Url = 'https://cmsweb-testbed.cern.ch/dbs/int/phys03/DBSReader'
# production DBS
migUrl='https://cmsweb.cern.ch/dbs/prod/phys03/DBSMigrate'
phy3Url = 'https://cmsweb.cern.ch/dbs/prod/phys03/DBSReader'

#datasets to migrate always come from global
globUrl='https://cmsweb.cern.ch/dbs/prod/global/DBSReader'
apiG   = DbsApi(url=globUrl)
apiP3  = DbsApi(url=phy3Url)
apiMig = DbsApi(url=migUrl)


In [25]:
# pick  a dataset
dataset='/Neutrino_E-10_gun/RunIISummer19ULPrePremix-UL17_106X_mc2017_realistic_v6-v1/PREMIX'

#find blocks
bList=[]
for dic in apiG.listBlocks(dataset=dataset):
    bList.append(dic['block_name'])
print(len(bList))


385


In [39]:
print('There are %d blocks in that dataset' % len(bList))

There are 385 blocks in that dataset


In [36]:
#submit migrations and collect their Id's
migrIds=[]
for b in bList:
    data= {'migration_url': globUrl, 'migration_input': b}
    result = apiMig.submitMigration(data)
    id = result.get("migration_details", {}).get("migration_request_id")
    migrIds.append(id)
doneMigrs=[]

In [70]:
print('created %d migration Ids' % len(migrIds))
print('Done migrations to skip in next iteration: %d' % len(doneMigrs))

created 385 migration Ids
Done migrations to skip in next iteration: 175


In [73]:
migrIds[0]

2879310

In [93]:
#check status, to be repeated until OK (state=2) or terminally failed (state=9)
# status values are:
#    0-request created;
#    1-migration in process;
#    2-migration successed;
#    3-migrationfailed, but has three chances to try;
#    9-migration Permanently failed.
# https://github.com/dmwm/DBS/blob/625525721c3df9a9b842c84ad0ac29f55180524b/Client/src/python/dbs/apis/dbsClient.py#L1489-L1492
failedMigs=[]
migTally={'created':0, 'ongoing':0, 'DONE':0, 'retrying':0, 'failed':0}
label={0:'created', 1:'ongoing', 2:'DONE', 3:'retrying', 9:'failed'}
for id in migrIds:
        if id in doneMigrs: continue
        status = apiMig.statusMigration(migration_rqst_id=id)
        if not status: continue # protection against stale list of migrIds
        state = status[0].get("migration_status")
        retry_count = status[0].get("retry_count")
        if state in [3,9]:
            print "migration id, state, retries : %d %d %s" % (id, state, retry_count)
        migTally[label[state]] += 1
        if state == 9 :
            failedMigs.append(id)
        if state == 2:
            doneMigrs.append(id)

print('Migration tally:\n%s' % migTally)
print('There are %d failed migrations' % len(failedMigs))
if failedMigs: print('first failed migration: %s' % failedMigs[0])
print('Done migrations to skip in next iteration: %d' % len(doneMigrs))
total = 0
for state in migTally.keys():
  total += migTally[state]
total = total - migTally['DONE'] + len(doneMigrs)
print('total = %d' % total)

Migration tally:
{'DONE': 56, 'retrying': 0, 'ongoing': 0, 'failed': 0, 'created': 0}
There are 0 failed migrations
Done migrations to skip in next iteration: 385
total = 385


In [94]:
# repeat previous count in case it went log in long output
print('Migration tally:\n%s' % migTally)
total = 0
print('There are %d failed migrations' % len(failedMigs))
if failedMigs: print('first failed migration: %s' % failedMigs[0])
print('Done migrations to skip in next iteration: %d' % len(doneMigrs))
for state in migTally.keys():
  total += migTally[state]
total = total - migTally['DONE'] + len(doneMigrs)
print('total = %d' % total)

Migration tally:
{'DONE': 56, 'retrying': 0, 'ongoing': 0, 'failed': 0, 'created': 0}
There are 0 failed migrations
Done migrations to skip in next iteration: 385
total = 385


In [34]:
# if there are old, failed migrations, need to remove them in order to
# submit them again, the first loop will simly collect their ID's
for mig in failedMigs:
    #mdic={'migration_rqst_id':mig}
    apiMig.removeMigration({'migration_rqst_id':mig})


In [ ]:
# now that failed migrations have remove you can go back to create the migrations from the block list.